In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import json

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.neural_network import BernoulliRBM
from sklearn.svm import SVC

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

f = open("C:/Users/krunifi/Documents/Work/MIDS/W207/Final/train.json", "r")
count = 0
labels = []
text = []
num_data = []

try:
    data = json.load(f)
    
    for item in data:
        
        # extract the outcome labels and text for request and title
        labels.append(int(item['requester_received_pizza']))
        #text.append(item['request_text'] + ' ' + item['request_title'])
        text.append(item['request_text'])
        
        # extract numeric data
        num = []
        num.append(item['number_of_downvotes_of_request_at_retrieval'])
        num.append(item['number_of_upvotes_of_request_at_retrieval'])
        num.append(item['request_number_of_comments_at_retrieval'])
        num.append(item['requester_account_age_in_days_at_request'])
        num.append(item['requester_days_since_first_post_on_raop_at_request'])
        num.append(item['requester_number_of_comments_at_retrieval'])
        num.append(item['requester_number_of_comments_in_raop_at_retrieval'])
        num.append(item['requester_number_of_posts_at_request'])
        num.append(item['requester_number_of_subreddits_at_request'])
        num.append(len(item['requester_subreddits_at_request']))
        num.append(item['unix_timestamp_of_request_utc'])
        num.append(len(item['request_text']))
        num.append(len(item['request_title']))
        num_data.append(num)

except Exception, e:
    print "error reading from file: %s" %e
    f.close()

f.close()
print len(labels)

4040


In [2]:
train_labels = np.array(labels[:3500])
train_data = np.array(text[:3500])
train_nums = np.array(num_data[:3500])

dev_labels = np.array(labels[3500:])
dev_data = np.array(text[3500:])
dev_nums = np.array(num_data[3500:])

In [3]:
# normalize the numerical data
for i in range(train_nums.shape[1]):
    max1 = np.mean(train_nums[:,i])
    max2 = np.mean(dev_nums[:,i])
    
    train_nums[:,i] = train_nums[:,i]/max1
    dev_nums[:,i] = dev_nums[:,i]/max2 
    
print dev_data[150], dev_labels[150]

print dev_nums.shape, train_nums.shape
print np.max(dev_nums[:,1])

http://m.youtube.com/?reason=8&amp;rdm=3676#/watch?v=V09Mt66O9_c&amp;desktop_uri=%2Fwatch%3Fv%3DV09Mt66O9_c 0
(540L, 13L) (3500L, 13L)
14.0579238529


In [4]:
def better_preprocessor(s):

    # remove links and replace with string "hyperlink"
    links = re.compile(u'\\bhttp\w+\\b')
    # remove non-alphanumerics
    non_alpha = re.compile(u'[\W_]+', re.UNICODE)
    # change all number sequences to single '0'
    num = re.compile(u'[0-9]+', re.UNICODE)
    # list of common English suffixes to remove
    suff = re.compile(u'(al|ance|ence|dom|ed|er|or|ism|ist|ity|ty|ment|ship|sion|ing|s|ly|ation|tion|able|ible|ate|en|ify|fy|ize|ise|ful|ic|ical|ious|ous|ish|ive)\\b', re.UNICODE)
    # list of common English prefixes to remove
    pre = re.compile(u'\\b(an|ante|anti|auto|circum|co|com|con|contra|de|dis|en|ex|extra|hyper|in|im|il|ir|inter|intra|macro|micro|mid|mis|mono|non|over|post|pre|pro|re|semi|sub|super|trans|tri|un)', re.UNICODE)
    # remove words with one or two characters only (treat as stopwords)
    stop = re.compile(u'\\b([\w]{1,2})\\b', re.UNICODE)

    # apply filters
    t = links.sub(u'hyperlink', s)
    t = non_alpha.sub(u' ', t)
    t = num.sub(u" 0 ", t)
    t = suff.sub(u' ', t)
    t = pre.sub(u' ', t)
    t = stop.sub(u' ', t)
    return t

In [5]:
# use vectorize text data and then train logistic regression
def trainLM():
    vectorizer = CountVectorizer(analyzer="word", ngram_range=[1,2])
    X = vectorizer.fit_transform(train_data)
    Y = vectorizer.transform(dev_data)
    vocab = vectorizer.get_feature_names()
    
    print X.shape, Y.shape
    
    lm = LogisticRegression()
    lm.fit(X, train_labels)
    pred_labels = lm.predict(Y)
    
    score = metrics.f1_score(dev_labels, pred_labels)
    print "base f1 =", score
    
    for row in range(lm.coef_.shape[0]):
        maxind = np.fabs(lm.coef_[row]).argsort()[-5:]
        for item in maxind:
            print vocab[item]
    
    # vectorize data using the better preprocessor
    better_vect = CountVectorizer(preprocessor=better_preprocessor)
    better_X = better_vect.fit_transform(train_data)
    better_Y = better_vect.transform(dev_data)
    vocab2 = better_vect.get_feature_names()
    
    # run logistic regression on this data and report f1 score
    better_lm = LogisticRegression()
    better_lm.fit(better_X, train_labels)
    better_labels = better_lm.predict(better_Y)
    print "Improved f1 =", metrics.f1_score(dev_labels, better_labels)
    
    for row in range(better_lm.coef_.shape[0]):
        #maxind2 = np.fabs(better_lm.coef_[row]).argsort()[-5:]
        maxind2 = better_lm.coef_[row].argsort()[-5:]
        for item in maxind2:
            print vocab2[item]
    
trainLM()

(3500, 107742) (540, 107742)
base f1 = 0.290155440415
friend
love pie
could make
when have
food for
Improved f1 = 0.27358490566
draft
aid
surpr
battle
NEED


In [6]:
# train a simple KNN model and check accuracy
# this uses the numerical data from json input
def trainKNN():
    # value for k from gridsearch output
    model = KNeighborsClassifier(n_neighbors=47)
    model.fit(train_nums, train_labels)
    pred_labels = model.predict(dev_nums)
    
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(pred_labels)
    print "%.03f" %accuracy

# run gridsearch to find best k value
def findK():
    # range is from an iterative process to narrow down
    params = {'n_neighbors': range(38,56,1)}
    search = GridSearchCV(KNeighborsClassifier(), params)
    search.fit(train_nums, train_labels)
    
    print "the best parameter is k=%f\n" %search.best_params_['n_neighbors']
    print "summary of all params:\n", search.grid_scores_

In [7]:
# train a simple BernoulliNB model and check accuracy
# this uses the numeric data captured from json input
def trainBern():
    bern = BernoulliNB(alpha=0)
    bern.fit(train_nums, train_labels)
    pred_labels = bern.predict(dev_nums)
    
    # display the accuracy
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(dev_labels)
    print "accuracy for BernoulliNB is %.03f" %accuracy

# run gridsearch to find parameters for NB model
def bernParams():
    params = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
    search = GridSearchCV(BernoulliNB(), params)
    search.fit(train_nums, train_labels)
    
    print "the best parameter is k=%f\n" %search.best_params_['alpha']
    print "summary of all params:\n", search.grid_scores_

In [8]:
trainKNN()
#findK()
trainBern()
#bernParams()

0.767
accuracy for BernoulliNB is 0.757


C:\Users\krunifi\Documents\Work\WinPython\WinPython-64bit-2.7.9.4\python-2.7.9.amd64\lib\site-packages\sklearn\naive_bayes.py:726: RuntimeWarning: divide by zero encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
C:\Users\krunifi\Documents\Work\WinPython\WinPython-64bit-2.7.9.4\python-2.7.9.amd64\lib\site-packages\sklearn\naive_bayes.py:729: RuntimeWarning: invalid value encountered in add
  jll += self.class_log_prior_ + neg_prob.sum(axis=1)


In [ ]:
# random forest classifier 
def trainRF():
    model = RandomForestClassifier()
    model.fit(train_nums, train_labels)
    pred_labels = model.predict(dev_nums)
    
    # display the accuracy
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(dev_labels)
    print "accuracy for RandomForest is %.03f" %accuracy
    
trainRF()